* **UJJWAL KUMAR - CS24RESCH11014**
* Content Multimedia Analysis

* checking gpu information

In [12]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jan 26 06:07:00 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              55W / 400W |  16059MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

* Seeing memory space

In [13]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


* installing requirement.txt

In [1]:
!pip install -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
 

* LLaVA model

In [2]:
'''
Notebook adapted from https://github.com/camenduru/LLaVA-colab and modified for ContextVLM
Author: Shounak Sural
Description: This notebook demonstrates how to use the LLaVA model for detecting contexts of interest from AV driving images.
'''

!git clone -b v1.0 https://github.com/camenduru/LLaVA
%cd LLaVA

!pip install -q transformers==4.36.2
!pip install -q gradio .

from transformers import AutoTokenizer, BitsAndBytesConfig
from llava.model import LlavaLlamaForCausalLM
import torch

model_path = "4bit/llava-v1.5-13b-3GB"
kwargs = {"device_map": "auto"}
kwargs['load_in_4bit'] = True
kwargs['quantization_config'] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)
model = LlavaLlamaForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, **kwargs)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

vision_tower = model.get_vision_tower()
if not vision_tower.is_loaded:
    vision_tower.load_model()
vision_tower.to(device='cuda')
image_processor = vision_tower.image_processor

import os
import requests
from PIL import Image
from io import BytesIO
from llava.conversation import conv_templates, SeparatorStyle
from llava.utils import disable_torch_init
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
from transformers import TextStreamer

def caption_image(image_file, prompt):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    disable_torch_init()
    conv_mode = "llava_v0"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
    inp = f"{roles[0]}: {prompt}"
    inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    raw_prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(raw_prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
    with torch.inference_mode():
      output_ids = model.generate(input_ids, images=image_tensor, do_sample=True, temperature=0.2,
                                  max_new_tokens=1024, use_cache=True, stopping_criteria=[stopping_criteria])
    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    conv.messages[-1][-1] = outputs
    output = outputs.rsplit('</s>', 1)[0]
    return image, output

Cloning into 'LLaVA'...
remote: Enumerating objects: 1960, done.
remote: Total 1960 (delta 0), reused 0 (delta 0), pack-reused 1960 (from 1)
Receiving objects: 100% (1960/1960), 13.60 MiB | 16.76 MiB/s, done.
Resolving deltas: 100% (1173/1173), done.
/content/LLaVA
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.9/809.9 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

pytorch_model-00001-of-00009.bin:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

* Connect to google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print("Hello Ujjwal")

Hello Ujjwal


* seeting path where i have detaset of vechicles on road

In [5]:
# self
file_names = os.listdir('/content/drive/MyDrive/DC_hand_annotated/pittsburgh')
sorted_file_names = sorted(file_names)

* performing question-answering

In [6]:
# self
file = open('/content/drive/MyDrive/context_results_pittsburgh.txt','a')

from tqdm import tqdm
output_list={}
for file_name in tqdm(sorted_file_names):
    try:
        question="Answer the following questions about this image. Generate output (yes/no) in JSON format with no additional text. Are we driving indoors?   Are we driving outdoors?  Is this during daytime?  Is this during nighttime?  Is this during sunny weather?  Is this during cloudy weather?  Is this during rainy weather?  Is this during snowy weather?  Is this during heavy fog?  Is this on a highway? Is this in a city? Does this have skyscrapers like in an urban canyon? Is this a rural area? Is this in a tunnel? Is this on a bridgeIs this in a parking lot? Is some construction work going on here? Is this a paved road? Are there lane markers on the ground? Is there heavy traffic? Is this in a sandstorm? Is this in an area with heavy tree cover? Is this on a dirt road? Is this beneath an underpass? Is this during twilight? "
        image, output = caption_image(f'/content/drive/MyDrive/DC_hand_annotated/pittsburgh/{file_name}', question)
        print(file_name,output)
        output_list[file_name]=output
        file.write(file_name+": "+output+"\n")
        # image
    except Exception as e:
        print(f"Error processing {file_name}: {str(e)}")
        continue

file.close()

  0%|          | 1/321 [00:21<1:55:35, 21.67s/it]

00000000.jpg {
"driving\_indoors": "no",
"driving\_outdoors": "yes",
"daytime": "yes",
"nighttime": "no",
"sunny\_weather": "yes",
"cloudy\_weather": "no",
"rainy\_weather": "no",
"snowy\_weather": "no",
"heavy\_fog": "no",
"highway": "yes",
"city": "no",
"urban\_canyon": "no",
"rural\_area": "yes",
"tunnel": "no",
"bridge": "no",
"parking\_lot": "no",
"construction\_work": "no",
"paved\_road": "yes",
"lane\_markers": "yes",
"heavy\_traffic": "no",
"sandstorm": "no",
"heavy\_tree\_cover": "no",
"dirt\_road": "no",
"underpass": "no",
"twilight": "no"
}


  1%|          | 2/321 [00:39<1:42:33, 19.29s/it]

00000009.jpg {
"indoors": "no",
"outdoors": "yes",
"daytime": "no",
"nighttime": "yes",
"sunny": "no",
"cloudy": "yes",
"rainy": "no",
"snowy": "no",
"heavy fog": "no",
"highway": "yes",
"city": "no",
"urban canyon": "no",
"rural": "yes",
"tunnel": "no",
"bridge": "no",
"parking lot": "no",
"construction work": "no",
"paved road": "yes",
"lane markers": "yes",
"heavy traffic": "no",
"sandstorm": "no",
"heavy tree cover": "no",
"dirt road": "no",
"underpass": "no",
"twilight": "no"
}


  1%|          | 3/321 [00:56<1:38:06, 18.51s/it]

...

proc_IMG_8149000043.jpg {
"indoors": "no",
"outdoors": "yes",
"daytime": "no",
"nighttime": "yes",
"sunny": "no",
"cloudy": "yes",
"rainy": "no",
"snowy": "no",
"heavy fog": "no",
"highway": "no",
"city": "yes",
"skyscrapers": "yes",
"rural": "no",
"tunnel": "no",
"bridge": "no",
"parking lot": "no",
"construction work": "no",
"paved road": "yes",
"lane markers": "no",
"heavy traffic": "no",
"sandstorm": "no",
"heavy tree cover": "no",
"dirt road": "no",
"underpass": "no",
"twilight": "no"
}


100%|█████████▉| 320/321 [1:36:18<00:17, 17.44s/it]

proc_IMG_8149000083.jpg {
"indoors": "no",
"outdoors": "yes",
"daytime": "no",
"nighttime": "yes",
"sunny": "no",
"cloudy": "no",
"rainy": "yes",
"snowy": "no",
"heavy fog": "yes",
"highway": "no",
"city": "yes",
"skyscrapers": "yes",
"rural": "no",
"tunnel": "no",
"bridge": "no",
"parking lot": "no",
"construction work": "no",
"paved road": "yes",
"lane markers": "no",
"heavy traffic": "no",
"sandstorm": "no",
"heavy tree cover": "no",
"dirt road": "no",
"underpass": "no",
"twilight": "no"
}


100%|██████████| 321/321 [1:36:35<00:00, 18.06s/it]

proc_IMG_8149000145.jpg {
"indoors": "no",
"outdoors": "yes",
"daytime": "no",
"nighttime": "yes",
"sunny": "no",
"cloudy": "yes",
"rainy": "no",
"snowy": "no",
"heavy fog": "no",
"highway": "no",
"city": "yes",
"skyscrapers": "yes",
"rural": "no",
"tunnel": "no",
"bridge": "no",
"parking lot": "no",
"construction work": "no",
"paved road": "yes",
"lane markers": "no",
"heavy traffic": "no",
"sandstorm": "no",
"heavy tree cover": "no",
"dirt road": "no",
"underpass": "no",
"twilight": "no"
}


* Downloading context results file

In [10]:
from google.colab import files
files.download("/content/drive/MyDrive/context_results_pittsburgh.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* downloading the generated LLaVA folder in to local

In [9]:
import shutil
from google.colab import files

# Compress the folder into a zip file
shutil.make_archive('/content/LLaVA', 'zip', '/content/LLaVA')

# Download the zip file
files.download('/content/LLaVA.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>